<a href="https://colab.research.google.com/github/SergeyB81/Portfolio-DS/blob/main/Car_price/model_analysis_and_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Загрузка данных**

In [8]:
! gdown --id 10pq7rvk_FtY-ulQ8ldVM4QAtaR6LWt-j

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10pq7rvk_FtY-ulQ8ldVM4QAtaR6LWt-j
To: /content/uk-used-cars.zip
100% 1.15M/1.15M [00:00<00:00, 13.0MB/s]


In [9]:
! unzip -o /content/uk-used-cars.zip -d /content/uk-used-cars

Archive:  /content/uk-used-cars.zip
  inflating: /content/uk-used-cars/audi.csv  
  inflating: /content/uk-used-cars/bmw.csv  
  inflating: /content/uk-used-cars/cclass.csv  
  inflating: /content/uk-used-cars/focus.csv  
  inflating: /content/uk-used-cars/ford.csv  
  inflating: /content/uk-used-cars/hyundi.csv  
  inflating: /content/uk-used-cars/merc.csv  
  inflating: /content/uk-used-cars/skoda.csv  
  inflating: /content/uk-used-cars/toyota.csv  
  inflating: /content/uk-used-cars/unclean cclass.csv  
  inflating: /content/uk-used-cars/unclean focus.csv  
  inflating: /content/uk-used-cars/vauxhall.csv  
  inflating: /content/uk-used-cars/vw.csv  


In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('/content/uk-used-cars/bmw.csv')

In [12]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


# **Обучающая, тестовая и валидационная выборка**

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train, test = train_test_split(df,train_size=0.6,random_state=42)

In [15]:
val, test = train_test_split(test,test_size=0.5,random_state=42)

# **Список фичей**



In [16]:
df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [17]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

# **Обучение модели**

In [18]:
! pip install catboost

In [19]:
from catboost import CatBoostRegressor

In [21]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}

In [22]:
model = CatBoostRegressor(**parameters)

In [23]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 68.6ms	remaining: 1m 8s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 1.36s	remaining: 12.2s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 1.96s	remaining: 7.79s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 2.56s	remaining: 5.94s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 3.13s	remaining: 4.68s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 3.75s	remaining: 3.73s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 4.32s	remaining: 2.87s
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 4.92s	remaining: 2.1s
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 5.51s	remaining: 1.37s
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 6.11s	remaining: 671ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 6.73s	remaining: 0us

bestTest = 0.06

# **Проверка на тестовой выборке**

In [24]:
test['price_pred'] = model.predict(test[X])

In [25]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [26]:
def error(y_true,y_pred):
  print(mean_absolute_error(y_true,y_pred))
  print(mean_absolute_percentage_error(y_true,y_pred))

In [29]:
error(test['price'],test['price_pred'])

1537.5600657036578
0.0724718966452533
